In [72]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
from string import punctuation
import string
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [73]:
train=  pd.read_csv(r'C:/Users/samri/OneDrive - Shri Vile Parle Kelavani Mandal/Desktop/sem 3 project/train.csv')
test = pd.read_csv('C:/Users/samri/OneDrive - Shri Vile Parle Kelavani Mandal/Desktop/sem 3 project/test.csv')

In [74]:
df= train.copy()
df.shape

(7613, 5)

In [75]:
# removing urls
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)
df['text']=df['text'].apply(lambda x : remove_URL(x))

In [76]:
# removing html tags
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
df['text']=df['text'].apply(lambda x : remove_html(x))

In [77]:
# removing emojis
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
df['text']=df['text'].apply(lambda x: remove_emoji(x))

In [78]:
# removing punctuations
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)
df['text']=df['text'].apply(lambda x : remove_punct(x))

In [79]:
# lemmatizing
import nltk
nltk.download('wordnet')
lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data
df['text'] = df['text'].apply(lambda x: lemmatizer_on_text(x))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\samri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [80]:
#stemming
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data

df['text'] = df['text'].apply(lambda x: stemming_on_text(x))
df['text'] = df['text'].str.lower()

In [81]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

df["tokens"] = df["text"].apply(tokenizer.tokenize)
df.head()

,id,keyword,location,text,target,tokens
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,"[our, deeds, are, the, reason, of, this, earth..."
1,4,NaN,NaN,forest fire near la ronge sask canada,1,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,all residents asked to shelter in place are be...,1,"[all, residents, asked, to, shelter, in, place..."
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1,"[13000, people, receive, wildfires, evacuation..."
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,"[just, got, sent, this, photo, from, ruby, ala..."


In [82]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

all_words = [word for tokens in df["tokens"] for word in tokens]
sentence_lengths = [len(tokens) for tokens in df["tokens"]]
VOCAB = sorted(list(set(all_words)))

In [83]:
list_labels = df["target"].tolist()

# Enter word2vec
Word2vec is a model that was pre-trained on a very large corpus, and provides embeddings that map words that are similar close to each other. A quick way to get a sentence embedding for our classifier, is to average word2vec scores of all words in our sentence.

In [38]:
#!pip install --upgrade numpy
#!pip install gensim==3.8.3
import gensim

# Load Google's pre-trained Word2Vec model.
word2vec_path = r"C:\Users\samri\Downloads\GoogleNews-vectors-negative300 (1).bin.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [84]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, df, generate_missing=False):
    embeddings = df['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [85]:
embeddings = get_word2vec_embeddings(word2vec, df)
from sklearn.model_selection import train_test_split
X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(embeddings, list_labels,
                                                                                        test_size=0.2, random_state=40)

In [86]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1


In [126]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
clf_w2v = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', random_state=40)
clf_w2v.fit(X_train_word2vec, y_train_word2vec)
y_predicted_word2vec = clf_w2v.predict(X_test_word2vec)

accuracy_word2vec, precision_word2vec, recall_word2vec, f1_word2vec = get_metrics(y_test_word2vec, y_predicted_word2vec)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_word2vec, precision_word2vec, 
                                                                       recall_word2vec, f1_word2vec))

accuracy = 0.804, precision = 0.803, recall = 0.804, f1 = 0.804


In [131]:
from sklearn.model_selection import GridSearchCV
# Hyperparameter tuning
param_grid = [
{'C' : [20, 25, 35, 40],
'solver' : ['lbfgs','liblinear','saga'],
'max_iter' : [1000,2500, 3500]}]
cv = GridSearchCV(clf_w2v, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1) #n_jobs = -1 means to use all the 
cv.fit(X_train_word2vec, y_train_word2vec)
cv.best_estimator_.fit(X_train_word2vec, y_train_word2vec)
y_pred = cv.best_estimator_.predict(X_test_word2vec)
accuracy_word2vec, precision_word2vec, recall_word2vec, f1_word2vec = get_metrics(y_test_word2vec, y_pred)
print(cv.best_estimator_)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_word2vec, precision_word2vec, 
                                                                       recall_word2vec, f1_word2vec))

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  1.6min finished


LogisticRegression(C=15, class_weight='balanced', max_iter=1000,
                   multi_class='multinomial', random_state=40)
accuracy = 0.802, precision = 0.802, recall = 0.802, f1 = 0.802


In [132]:
#SVM
from sklearn import svm  
clf = svm.SVC(kernel='rbf') 
clf.fit(X_train_word2vec, y_train_word2vec)
y_pred1 = clf.predict(X_test_word2vec)

accuracy, precision, recall, f1 = get_metrics(y_test_word2vec, y_pred1)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.830, precision = 0.836, recall = 0.830, f1 = 0.827


In [134]:
# hyperparameter tuning
from scipy.stats import reciprocal, uniform
param_distributions = {"gamma":reciprocal(0.001,0.1), "C":uniform(1,10)}
from sklearn.model_selection import RandomizedSearchCV

cv = RandomizedSearchCV(clf, param_distributions, n_iter = 40, verbose = 1, cv = 3)
cv.fit(X_train_word2vec, y_train_word2vec)
cv.best_estimator_.fit(X_train_word2vec, y_train_word2vec)
y_pred1 = cv.best_estimator_.predict(X_test_word2vec)
accuracy, precision, recall, f1 = get_metrics(y_test_word2vec, y_pred1)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 14.5min finished


accuracy = 0.808, precision = 0.811, recall = 0.808, f1 = 0.805


In [98]:
#naive bayes
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

bernoulliNBModel = BernoulliNB(alpha=0.1)
bernoulliNBModel.fit(X_train_word2vec, y_train_word2vec)
y_pred1 = bernoulliNBModel.predict(X_test_word2vec)
accuracy, precision, recall, f1 = get_metrics(y_test_word2vec, y_pred1)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.747, precision = 0.749, recall = 0.747, f1 = 0.748


In [107]:
param_grid_nb = {'alpha': (0.1, 0.2, 0.3,0.5,1,5,10)}

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
nbModel_grid = GridSearchCV(estimator=BernoulliNB(), param_grid=param_grid_nb, verbose=1, cv=4, n_jobs=-1)
nbModel_grid.fit(X_train_word2vec, y_train_word2vec)
y_pred1 = nbModel_grid.predict(X_test_word2vec)
accuracy, precision, recall, f1 = get_metrics(y_test_word2vec, y_pred1)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

Fitting 4 folds for each of 7 candidates, totalling 28 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


accuracy = 0.747, precision = 0.749, recall = 0.747, f1 = 0.748


[Parallel(n_jobs=-1)]: Done  28 out of  28 | elapsed:    6.9s finished


In [108]:
#MultinomialNBModel
# negative values

ValueError: Negative values in data passed to MultinomialNB (input X)

In [115]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(max_features=5, n_estimators=100)
rf.fit(X_train_word2vec, y_train_word2vec)
y_pred1 = rf.predict(X_test_word2vec)
accuracy, precision, recall, f1 = get_metrics(y_test_word2vec, y_pred1)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.796, precision = 0.801, recall = 0.796, f1 = 0.792


In [121]:
param_grid = {
    'n_estimators': [80,120,140],
    'max_depth': [3,4,5,6,7]
}
# Perform grid search on n_estimators and max_depth
gridsearch = GridSearchCV(estimator = rf, param_grid=param_grid, verbose=True, scoring='accuracy')
gridsearch.fit(X_train_word2vec, y_train_word2vec)
gridsearch.best_estimator_.fit(X_train_word2vec, y_train_word2vec)
y_pred1 = rf.predict(X_test_word2vec)
accuracy, precision, recall, f1 = get_metrics(y_test_word2vec, y_pred1)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  1.8min finished


accuracy = 0.796, precision = 0.801, recall = 0.796, f1 = 0.792


In [109]:
#Ridge Regression
clf = linear_model.RidgeClassifier()
clf.fit(X_train_word2vec, y_train_word2vec)
y_pred1 = clf.predict(X_test_word2vec)
accuracy, precision, recall, f1 = get_metrics(y_test_word2vec, y_pred1)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.794, precision = 0.795, recall = 0.794, f1 = 0.792


In [110]:
param_grid = {"alpha":(0.1,0.5,1,3,5)}
cv = GridSearchCV(clf, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)
cv.fit(X_train_word2vec, y_train_word2vec)
cv.best_estimator_.fit(X_train_word2vec, y_train_word2vec)
y_pred = cv.best_estimator_.predict(X_test_word2vec)
accuracy, precision, recall, f1 = get_metrics(y_test_word2vec, y_pred)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


accuracy = 0.793, precision = 0.794, recall = 0.793, f1 = 0.790


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    4.2s finished


In [124]:
#Xgboost
from xgboost import XGBClassifier
model = XGBClassifier(use_label_encoder=False)
model.fit(X_train_word2vec, y_train_word2vec)
y_pred1 = model.predict(X_test_word2vec)
accuracy, precision, recall, f1 = get_metrics(y_test_word2vec, y_pred1)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

[18:26:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy = 0.800, precision = 0.801, recall = 0.800, f1 = 0.798


In [125]:
from sklearn.model_selection import RandomizedSearchCV
params={
 "learning_rate"    : [0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 4, 5, 6, 8, 10, 12],
 "min_child_weight" : [ 2,4,3, 5],
 "gamma"            : [ 0.1, 0.2 , 0.3],
 "colsample_bytree" : [ 0.4, 0.5]}
random_search=RandomizedSearchCV(model,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

random_search.fit(X_train_word2vec, y_train_word2vec)
random_search.best_estimator_.fit(X_train_word2vec, y_train_word2vec)
y_pred = random_search.best_estimator_.predict(X_test_word2vec)
accuracy, precision, recall, f1 = get_metrics(y_test_word2vec, y_pred)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  2.1min remaining:   39.3s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.4min finished


[18:29:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:29:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy = 0.810, precision = 0.812, recall = 0.810, f1 = 0.808


In [135]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=35)
knn.fit(X_train_word2vec, y_train_word2vec)
y_pred = knn.predict(X_test_word2vec)
accuracy, precision, recall, f1 = get_metrics(y_test_word2vec, y_pred)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.783, precision = 0.789, recall = 0.783, f1 = 0.779


In [136]:
#hyperparameter tuning
k=np.random.randint(1,50,60)
params={'n_neighbors': k}
random_search=RandomizedSearchCV(knn,params,n_iter=40,cv=4,n_jobs=-1,verbose=1)
random_search.fit(X_train_word2vec, y_train_word2vec)
random_search.best_estimator_.fit(X_train_word2vec, y_train_word2vec)
y_pred = random_search.predict(X_test_word2vec)
accuracy, precision, recall, f1 = get_metrics(y_test_word2vec, y_pred)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

Fitting 4 folds for each of 40 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  3.1min finished


accuracy = 0.791, precision = 0.800, recall = 0.791, f1 = 0.785


# CNN

In [42]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH= 35
VOCAB_SIZE= len(VOCAB)

VALIDATION_SPLIT=.2
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(df["text"].tolist())
sequences = tokenizer.texts_to_sequences(df["text"].tolist())

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

cnn_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(df["target"]))

indices = np.arange(cnn_data.shape[0])
np.random.shuffle(indices)
cnn_data = cnn_data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * cnn_data.shape[0])

embedding_weights = np.zeros((len(word_index)+1, EMBEDDING_DIM))
for word,index in word_index.items():
    embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(embedding_weights.shape)

Found 18104 unique tokens.
(18105, 300)


In [52]:
print(cnn_data)

[[    0     0     0 ... 11345   402   280]
 [    0     0     0 ...    70   324    90]
 [    0     0     0 ...  7597   723  2944]
 ...
 [    0     0     0 ...    16    79  1101]
 [    0     0     0 ...     2  4057  5186]
 [    0     0     0 ...   589    78    21]]


In [53]:
cnn_data.shape

(7613, 35)

In [55]:
y=df['target']

In [56]:
len(cnn_data),y.shape

(7613, (7613,))

In [28]:
from keras.layers import Dense, Input, Flatten, Dropout, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model

def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index, trainable=False, extra_conv=True):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    #x = Dropout(0.5)(x)

    preds = Dense(labels_index, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])

    return model

In [30]:
x_train = cnn_data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = cnn_data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

model = ConvNet(embedding_weights, MAX_SEQUENCE_LENGTH, len(word_index)+1, EMBEDDING_DIM, 
                len(list(df["target"].unique())), False)

model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=128)

Epoch 1/10
48/48 [==============================] - 6s 115ms/step - loss: 0.6814 - acc: 0.6204 - val_loss: 0.5572 - val_acc: 0.7227
Epoch 2/10
48/48 [==============================] - 5s 107ms/step - loss: 0.4877 - acc: 0.7702 - val_loss: 0.4786 - val_acc: 0.7707
Epoch 3/10
48/48 [==============================] - 5s 111ms/step - loss: 0.4231 - acc: 0.8084 - val_loss: 0.4356 - val_acc: 0.8095
Epoch 4/10
48/48 [==============================] - 5s 109ms/step - loss: 0.3680 - acc: 0.8422 - val_loss: 0.4546 - val_acc: 0.7917
Epoch 5/10
48/48 [==============================] - 5s 108ms/step - loss: 0.3224 - acc: 0.8655 - val_loss: 0.4551 - val_acc: 0.8022
Epoch 6/10
48/48 [==============================] - 5s 114ms/step - loss: 0.2706 - acc: 0.8898 - val_loss: 0.4717 - val_acc: 0.7989
Epoch 7/10
48/48 [==============================] - 5s 111ms/step - loss: 0.2239 - acc: 0.9159 - val_loss: 0.5162 - val_acc: 0.7996
Epoch 8/10
48/48 [==============================] - 6s 116ms/step - loss: 0.

In [62]:
'''y_pred1 = model.predict(y_train)
accuracy, precision, recall, f1 = get_metrics(y_val, y_pred1)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))'''

'y_pred1 = model.predict(y_train)\naccuracy, precision, recall, f1 = get_metrics(y_val, y_pred1)\nprint("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))'

In [67]:
embedding_vector_features=300
model=Sequential()
model.add(Embedding(len(word_index)+1, embedding_vector_features,weights=[embedding_weights], input_length= MAX_SEQUENCE_LENGTH,trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 35, 300)           5431500   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 35, 300)           0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 64)                93440     
_________________________________________________________________
dense_24 (Dense)             (None, 2)                 130       
Total params: 5,525,070
Trainable params: 93,570
Non-trainable params: 5,431,500
_________________________________________________________________
None


In [66]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense, Embedding, Input, Flatten, Dropout, SpatialDropout1D


#Embedding(len(word_index)+1,EMBEDDING_DIM ,weights=[embedding_weights],input_length=MAX_SEQUENCE_LENGTH,trainable=False)
# for units in [128,128,64,32]:

embedding_vector_features=300
model=Sequential()
model.add(Embedding(len(word_index)+1,EMBEDDING_DIM ,weights=[embedding_weights],input_length=MAX_SEQUENCE_LENGTH,trainable=False))
model.add(LSTM(128,input_shape=(cnn_data.shape),activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 35, 300)           5431500   
_________________________________________________________________
lstm_15 (LSTM)               (None, 35, 128)           219648    
_________________________________________________________________
dropout_28 (Dropout)         (None, 35, 128)           0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_29 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_30 (Dropout)         (None, 64)              

In [68]:
import numpy as np
X_final=np.array(cnn_data)
y_final=np.array(y)

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X_final, y_final, test_size=0.3, random_state=40)

In [70]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=10, batch_size=50)

Epoch 1/10
107/107 [==============================] - 12s 87ms/step - loss: 0.6903 - accuracy: 0.5571 - val_loss: 0.6837 - val_accuracy: 0.5749
Epoch 2/10
107/107 [==============================] - 9s 83ms/step - loss: 0.6823 - accuracy: 0.5691 - val_loss: 0.6853 - val_accuracy: 0.5762
Epoch 3/10
107/107 [==============================] - 9s 83ms/step - loss: 0.6773 - accuracy: 0.5708 - val_loss: 0.6847 - val_accuracy: 0.5705
Epoch 4/10
107/107 [==============================] - 9s 84ms/step - loss: 0.6747 - accuracy: 0.5802 - val_loss: 0.6889 - val_accuracy: 0.5740
Epoch 5/10
107/107 [==============================] - 10s 90ms/step - loss: 0.6673 - accuracy: 0.5937 - val_loss: 0.6893 - val_accuracy: 0.5731
Epoch 6/10
107/107 [==============================] - 10s 97ms/step - loss: 0.6573 - accuracy: 0.6039 - val_loss: 0.6998 - val_accuracy: 0.5525
Epoch 7/10
107/107 [==============================] - 9s 86ms/step - loss: 0.6473 - accuracy: 0.6198 - val_loss: 0.7065 - val_accuracy: 0.5

# CNN FROM NET

In [66]:
def create_tokenizer(lines):
     tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
vocab_size = len(tokenizer.word_index) + 1

In [67]:
def define_model(length, vocab_size):

    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [74]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model
from numpy import array

In [79]:
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded
trainX = encode_text(tokenizer, train['text'], length)

In [82]:
def max_length(lines):
    return max([len(s.split()) for s in lines])
length = max_length(train['text'])
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX,trainX],array(y_train), epochs=10, batch_size=16)
# save the model
#model.save('model.h5')

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 33)]         0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 33)]         0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           [(None, 33)]         0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 33, 100)      100         input_19[0][0]                   
____________________________________________________________________________________________

ValueError: Data cardinality is ambiguous:
  x sizes: 7613, 7613, 7613
  y sizes: 5000
Make sure all arrays contain the same number of samples.